In [1]:
import numpy as np
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from sklearn import metrics


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'haiku'


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(30, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, 256)
        self.conv4 = GCNConv(256, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = global_max_pool(x, data.batch)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
def custom_collate(batch):
    data_list, target_list = zip(*batch)
    batch_data = Batch.from_data_list(data_list)
    batch_target = torch.stack(target_list)
    return batch_data, batch_target

In [4]:
def calculate_statistics(group):
    r2_train = group['r2_train']
    r2_test = group['r2_test']
    return pd.Series({
        'r2_train_mean': np.mean(r2_train),
        'r2_train_std': np.std(r2_train, ddof=0),
        'r2_test_mean': np.mean(r2_test),
        'r2_test_std': np.std(r2_test, ddof=0)
    })

In [5]:
torch.manual_seed(0)

scaler = StandardScaler()

first_epochs = 50
second_epochs = 200
second_lr = 9e-4
second_wd = 1e-3

results = []
for random_state in range(10):
    torch.manual_seed(0)
    print(f'Random State: {random_state}')
    
    for d in ["AI", "AI2", "Random", "Human"]:
        torch.manual_seed(0)
        print('Source : ', d)
        dataset = d
        
        for c in ["Yield_CO"]:
            torch.manual_seed(0)
            df = pd.read_csv('data_Real/data_real.csv')
            smiles = df["SMILES"]
            featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
            X = featurizer.featurize(smiles)
            
            y = df[c]
            y = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()
            
            data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.5, random_state=random_state)
            target_train = torch.tensor(target_train, dtype=torch.float32)
            target_test = torch.tensor(target_test, dtype=torch.float32)

            data_train_list = []
            for graph_data in data_train:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_train_list.append(data)

            data_test_list = []
            for graph_data in data_test:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_test_list.append(data)

            train_loader = DataLoader(list(zip(data_train_list, target_train)), batch_size=len(data_train_list), collate_fn=custom_collate)
            test_loader = DataLoader(list(zip(data_test_list, target_test)), batch_size=len(data_test_list), collate_fn=custom_collate)

            model = Net()
            model.load_state_dict(torch.load(f'data_{dataset}/model_{dataset}_sc.pth'))
            model.fc3 = nn.Linear(128, 1)
        
            # Freeze all layers
            for param in model.parameters():
                param.requires_grad = False
        
            model.train()
            optimizer = torch.optim.Adam(model.parameters())
            criterion = nn.MSELoss()
            
            device = torch.device('cpu')
            model.to(device)

            # Initial training with frozen layers (no optimizer step needed)
            for epoch in range(first_epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    with torch.no_grad():
                        out = model(data)
                        loss = criterion(out, target.view(-1, 1))

            # Unfreeze fully connected layers for fine-tuning
            for param in model.fc1.parameters():
                param.requires_grad = True
            for param in model.fc2.parameters():
                param.requires_grad = True
            for param in model.fc3.parameters():
                param.requires_grad = True

            # Redefine the optimizer to include only the parameters that require gradients
            optimizer = torch.optim.Adam(model.parameters(), lr=second_lr, weight_decay=second_wd)

            # Fine-tuning
            for epoch in range(second_epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    optimizer.zero_grad()
                    out = model(data)
                    loss = criterion(out, target.view(-1, 1))
                    loss.backward()
                    optimizer.step()

            model.eval()
            pred_train = []
            for data, target in train_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_train.append(out.cpu().numpy())
            pred_train = np.concatenate(pred_train)

            pred_test = []
            for data, target in test_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_test.append(out.cpu().numpy())
            pred_test = np.concatenate(pred_test)

            pred_train = scaler.inverse_transform(pred_train)
            pred_test = scaler.inverse_transform(pred_test)
            target_train = scaler.inverse_transform(target_train.numpy().reshape(-1, 1)).flatten()
            target_test = scaler.inverse_transform(target_test.numpy().reshape(-1, 1)).flatten()

            r2_train_score = metrics.r2_score(target_train, pred_train)
            r2_test_score = metrics.r2_score(target_test, pred_test)

            results.append({'source': d, 'target': c, 'r2_train': r2_train_score, 'r2_test': r2_test_score})
            print(f'R2 test for {c} with random state {random_state}: {r2_test_score}')

results_df = pd.DataFrame(results)
gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()
gen_results

Random State: 0
Source :  AI
R2 test for Yield_CO with random state 0: 0.756309449672699
Source :  AI2
R2 test for Yield_CO with random state 0: 0.7531410455703735
Source :  Random
R2 test for Yield_CO with random state 0: 0.8188301920890808
Source :  Human
R2 test for Yield_CO with random state 0: 0.6971414089202881
Random State: 1
Source :  AI
R2 test for Yield_CO with random state 1: 0.7084928154945374
Source :  AI2
R2 test for Yield_CO with random state 1: 0.7425848841667175
Source :  Random
R2 test for Yield_CO with random state 1: 0.8047307729721069
Source :  Human
R2 test for Yield_CO with random state 1: 0.7165088653564453
Random State: 2
Source :  AI
R2 test for Yield_CO with random state 2: 0.5206757187843323
Source :  AI2
R2 test for Yield_CO with random state 2: 0.5358211994171143
Source :  Random
R2 test for Yield_CO with random state 2: 0.6689878702163696
Source :  Human
R2 test for Yield_CO with random state 2: 0.6986278891563416
Random State: 3
Source :  AI
R2 test for 

C:\Users\noton\AppData\Local\Temp\ipykernel_15812\445282645.py:128: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()


,source,target,r2_train_mean,r2_train_std,r2_test_mean,r2_test_std
0,AI,Yield_CO,0.975708,0.007445,0.706232,0.097122
1,AI2,Yield_CO,0.974661,0.012711,0.714057,0.102845
2,Human,Yield_CO,0.960409,0.020851,0.713548,0.054499
3,Random,Yield_CO,0.967922,0.014026,0.765419,0.079950


In [6]:
gen_results.to_csv(f'result/Yield/twostep_{first_epochs}_{second_epochs}.csv')